In [33]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

# эти штуки нужны для ворд ту века, чтобы обучить модель на вики дате, вики дату надо запихать в эти функции
# а потом надо взять данные и их тоже запихать и тоооолько потом уже пытаться делать векторы и что то там себе оценивать!!!

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]
    return ' '.join(tokens)

# вот эта штука нужна для уже предобученной модели русвек, прежде чем пихать в нее данные,
# их надо запихать в функцию normalize_mystem()
mapping = {}

for line in open('ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud
    
from pymystem3 import Mystem
m = Mystem()

def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)
    return tokens

In [34]:
# Тут я буду обучать модель самостоятельно word2vec, не на парафразах! это точно не они
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api
# Download dataset

file = open('wiki_data.txt').read().splitlines()
data_norm = [normalize(text) for text in file]
data_norm = [text for text in data_norm if text]
# Train Word2Vec model. Defaults result vector size = 100
# используй все ядра и отсеки слова с частотой 1
model = Word2Vec([text.split() for text in data_norm], min_count = 10, workers=-1)
# Save and Load Model
# model.save('newmodel')
# model = Word2Vec.load('newmodel')

In [22]:
#import gensim
# берем какую то вторую модель, которую я скачала по ссылке из домашки
#rusvec = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True)

In [35]:
# теперь надо взять тексты, открываем парафразы
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [36]:
'''data['text_1_norm'] = data['text_1'].apply(normalize_mystem)
data['text_2_norm'] = data['text_2'].apply(normalize_mystem)'''# rusvec
# word2vec
data['text_1_norm'] = data['text_1'].apply(normalize).apply(tokenize)
data['text_2_norm'] = data['text_2'].apply(normalize).apply(tokenize)

In [1]:
#data_norm

In [ ]:
'''text_test = data['text_1_norm'].values[0]
text_test'''

In [37]:
'''def get_embedding(text, model, dim):
    text = text.split()
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
            print(vectors[i])
        except (KeyError, ValueError):
            print('Error')
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector'''

def get_embedding(text, model, dim):
    
    #text = text.split()
    #print(text)
    new_text = []
    for word in text:
        if word in model:
            new_text.append(word)
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(new_text)
    #print(words)
    total = len(new_text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        v = model[word]
        #print(v,'vvvvvv')
        vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        #print(vectors[i])
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        #print('else')
        vector = np.zeros((dim))
    
    return vector

#get_embedding(text_test, model, dim=100)

In [38]:
# делаем для ворд ту века
# это размерность вектора
dim = 100
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))
#print(model.wv.index2word)
for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, model, dim=100)
    #print( X_text_1_w2v[i],'X_text_1_w2v[i]')
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, model, dim=100)

/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [4]:
'''X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)
y = data['label'].values
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)'''

"X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)\ny = data['label'].values\ntrain_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)"

In [40]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, train_X, train_y, cv=5)
print(scores)
#array([0.96..., 1. , 0.96..., 0.96..., 1. ]) что то примерно такое должно получится

[0.4095941 0.4095941 0.4095941 0.4095941 0.4095941]


In [41]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.41 accuracy with a standard deviation of 0.00


In [2]:
# теперь делаем для второй модели
'''dim = 300
X_text_1_rusvec = np.zeros((len(data['text_1_norm']), dim))
X_text_2_rusvec = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_rusvec[i] = get_embedding(text, rusvec, dim=300)
    print(X_text_1_rusvec[i])
    
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_rusvec[i] = get_embedding(text, rusvec, dim=300)'''
    

"dim = 300\nX_text_1_rusvec = np.zeros((len(data['text_1_norm']), dim))\nX_text_2_rusvec = np.zeros((len(data['text_2_norm']), dim))\n\nfor i, text in enumerate(data['text_1_norm'].values):\n    X_text_1_rusvec[i] = get_embedding(text, rusvec, dim=300)\n    print(X_text_1_rusvec[i])\n    \n    \nfor i, text in enumerate(data['text_2_norm'].values):\n    X_text_2_rusvec[i] = get_embedding(text, rusvec, dim=300)"

In [32]:
'''from sklearn.model_selection import cross_val_score
from sklearn import svm
y = data['label'].values
X_text_rusvec = np.concatenate([X_text_1_rusvec, X_text_2_rusvec], axis=1)
train_X, valid_X, train_y, valid_y = train_test_split(X_text_rusvec, y,random_state=1)
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, train_X, train_y, cv=5)
scores
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))'''

0.45 accuracy with a standard deviation of 0.01
